* [네이버 오픈 API 목록 - INTRO](https://developers.naver.com/products/intro/plan/plan.md)
* [애플리케이션 - NAVER Developers](https://developers.naver.com/apps/#/list)

In [ ]:
import os
import requests
import pandas as pd
import time
import logging
from tqdm import tqdm
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# 환경변수 로드
load_dotenv()

In [ ]:
# 로깅 설정
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f'shopping_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

# 데이터 수집 진행상황 모니터링 클래스
class DataCollectionMonitor:
    def __init__(self, total_requests):
        self.total_requests = total_requests
        self.current_request = 0
        self.start_time = time.time()
        self.progress_bar = tqdm(total=total_requests, desc="데이터 수집 진행률")
        
    def update(self):
        self.current_request += 1
        self.progress_bar.update(1)
        
    def get_progress(self):
        return (self.current_request / self.total_requests) * 100
    
    def get_elapsed_time(self):
        return time.time() - self.start_time
    
    def close(self):
        self.progress_bar.close()

# API 요청
def fetch_data(client_id, client_secret, url, categories, start_date, end_date, time_unit, devices, genders, ages, logger):
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret,
        "Content-Type": "application/json"
    }
    all_results = []
    
    # 총 요청 횟수 계산
    total_requests = len(devices) * len(genders) * len(ages) * len(categories)
    monitor = DataCollectionMonitor(total_requests)
    
    for device in devices:
        for gender in genders:
            for age_group in ages:
                for category in categories:
                    logger.info(f"수집 중: {device}-{gender}-{age_group}-{category['name']}")
                    
                    body = {
                        "startDate": start_date,
                        "endDate": end_date,
                        "timeUnit": time_unit,
                        "category": [category],
                        "device": device,
                        "gender": gender,
                        "ages": [age_group]
                    }
                    
                    try:
                        response = requests.post(url, headers=headers, json=body)
                        if response.status_code == 200:
                            data = response.json()
                            for category_data in data["results"]:
                                for data_point in category_data["data"]:
                                    all_results.append({
                                        "date": data_point["period"],
                                        "category": category["name"],
                                        "device": device,
                                        "gender": gender,
                                        "age_group": age_group,
                                        "ratio": data_point["ratio"]
                                    })
                            logger.info(f"성공: {category['name']} 데이터 수집 완료")
                        else:
                            logger.error(f"API 요청 실패: {category['name']}, 상태 코드: {response.status_code}")
                    
                    except Exception as e:
                        logger.error(f"에러 발생: {str(e)}")
                    
                    monitor.update()
                    
                    # API 호출 간격 조절 (초당 요청 제한 고려)
                    time.sleep(0.1)
    
    monitor.close()
    logger.info(f"총 소요시간: {monitor.get_elapsed_time():.2f}초")
    
    return pd.DataFrame(all_results)

In [ ]:
# 로거 설정
logger = setup_logging()
logger.info("프로그램 시작")

# 설정값 정의
CATEGORIES = [
    {"name": "패션의류", "param": ["50000000"]},
    {"name": "화장품/미용", "param": ["50000002"]},
    {"name": "디지털/가전", "param": ["50000003"]},
    {"name": "가구/인테리어", "param": ["50000004"]},
    {"name": "출산/육아", "param": ["50000005"]},
    {"name": "스포츠/레저", "param": ["50000006"]},
    {"name": "생활/건강", "param": ["50000007"]},
    {"name": "여가/생활편의", "param": ["50000008"]},
    {"name": "식품", "param": ["50000009"]},
    {"name": "도서", "param": ["50000010"]}
]

START_DATE = "2024-01-01"
END_DATE = "2024-12-15"
TIME_UNIT = "date"
DEVICES = ["pc", "mobile"]
GENDERS = ["f", "m"]
AGES = ["10", "20", "30", "40", "50", "60"]


# 환경변수에서 API 키 로드
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
URL = "https://openapi.naver.com/v1/datalab/shopping/categories"

if not CLIENT_ID or not CLIENT_SECRET:
    logger.error("CLIENT_ID 또는 CLIENT_SECRET이 설정되지 않았습니다. .env 파일을 확인하세요.")
    exit(1)


try:
    # 데이터 수집
    logger.info("데이터 수집 시작")
    df = fetch_data(
        CLIENT_ID, CLIENT_SECRET, URL, CATEGORIES,
        START_DATE, END_DATE, TIME_UNIT,
        DEVICES, GENDERS, AGES,
        logger
    )
    
    if not df.empty:
        # 원본 데이터 저장
        df.to_csv("raw_shopping_data.csv", index=False)
        logger.info("원본 데이터 저장 완료")
        
        # 전처리된 데이터 저장
        df.to_csv("processed_shopping_data.csv", index=False)
        logger.info("전처리된 데이터 저장 완료")
        
    else:
        logger.error("수집된 데이터가 없습니다.")
        
except Exception as e:
    logger.error(f"프로그램 실행 중 오류 발생: {str(e)}")

finally:
    logger.info("프로그램 종료")

In [ ]:

        # 데이터 전처리
        df = preprocess_data(df, logger)

In [ ]:
df = pd.read_csv('raw_shopping_data.csv')
df

In [ ]:
# 데이터 전처리
logger.info("데이터 전처리 시작")

# 결측치 확인
missing_values = df.isnull().sum()
logger.info(f"결측치 현황:\n{missing_values}")

# 이상치 확인 (ratio 값이 0-100 범위를 벗어나는 경우)
outliers = df[~df['ratio'].between(0, 100)]
logger.info(f"이상치 개수: {len(outliers)}")

# 데이터 타입 변환
df['date'] = pd.to_datetime(df['date'])
df['age_group'] = df['age_group'].astype(int)

# 기초 통계량 출력
logger.info("\n기초 통계량:")
logger.info(f"\n{df.describe()}")

In [ ]:
# 데이터 타입 변환 및 날짜 관련 컬럼 생성
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['dayofweek'] = df['date'].dt.dayofweek

df['dayofweekname'] = df['dayofweek'].map(lambda x: '월화수목금토일'[x])

In [ ]:
df.describe()

In [ ]:
# 카테고리별 평균 비율 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='category', y='ratio', data=df, errorbar=None)
plt.title('카테고리별 평균 비율')

plt.figure(figsize=(10, 6))
sns.barplot(x='category', y='ratio', data=df, errorbar=None, hue='gender')
plt.title('카테고리별 평균 비율')

# 성별에 따른 비율 분포 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='gender', y='ratio', data=df)
plt.title('성별에 따른 비율 분포')

# 연령대별 평균 비율 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='ratio', data=df, errorbar=None)
plt.title('연령대별 평균 비율')

plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='ratio', data=df, errorbar=None, hue='gender')
plt.title('연령대별 평균 비율')

# 월별 평균 비율 시각화
plt.figure(figsize=(10, 6))
sns.lineplot(x='month', y='ratio', data=df, marker='o')
plt.title('월별 평균 비율 추이')

# 요일별 평균 비율 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='dayofweekname', y='ratio', data=df, errorbar=None,
            order=list('월화수목금토일'))
plt.title('요일별 평균 비율')


In [ ]:
sns.catplot(x='age_group', y='ratio', data=df, errorbar=None, 
            kind='bar', hue='gender', col='category', col_wrap=4)
plt.title('카테고리별 평균 비율')